In [16]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from torch.utils.data import Dataset
from sklearn.preprocessing import OneHotEncoder
import torch

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     print(dirname)
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# We have to predict one of the five lumbar spine degenerative conditions, and where it happens (L1/L2,...) + the severity

In [17]:
%pip install einops

Note: you may need to restart the kernel to use updated packages.


In [3]:
train_path = '/kaggle/input/rsna-2024-lumbar-spine-degenerative-classification/train.csv'

train = pd.read_csv(train_path).dropna()
train.head()
x=train.iloc[:, 1:]
label_enc = OneHotEncoder()
sparse_matrix = label_enc.fit_transform(x)
feature_names = label_enc.get_feature_names_out(train.iloc[:, 1:].columns)
df = pd.DataFrame.sparse.from_spmatrix(sparse_matrix, columns=feature_names)
print("One-Hot Encoded Feature Names:")
print(len(feature_names))
print(feature_names)
# print(df)

One-Hot Encoded Feature Names:
75
['spinal_canal_stenosis_l1_l2_Moderate'
 'spinal_canal_stenosis_l1_l2_Normal/Mild'
 'spinal_canal_stenosis_l1_l2_Severe'
 'spinal_canal_stenosis_l2_l3_Moderate'
 'spinal_canal_stenosis_l2_l3_Normal/Mild'
 'spinal_canal_stenosis_l2_l3_Severe'
 'spinal_canal_stenosis_l3_l4_Moderate'
 'spinal_canal_stenosis_l3_l4_Normal/Mild'
 'spinal_canal_stenosis_l3_l4_Severe'
 'spinal_canal_stenosis_l4_l5_Moderate'
 'spinal_canal_stenosis_l4_l5_Normal/Mild'
 'spinal_canal_stenosis_l4_l5_Severe'
 'spinal_canal_stenosis_l5_s1_Moderate'
 'spinal_canal_stenosis_l5_s1_Normal/Mild'
 'spinal_canal_stenosis_l5_s1_Severe'
 'left_neural_foraminal_narrowing_l1_l2_Moderate'
 'left_neural_foraminal_narrowing_l1_l2_Normal/Mild'
 'left_neural_foraminal_narrowing_l1_l2_Severe'
 'left_neural_foraminal_narrowing_l2_l3_Moderate'
 'left_neural_foraminal_narrowing_l2_l3_Normal/Mild'
 'left_neural_foraminal_narrowing_l2_l3_Severe'
 'left_neural_foraminal_narrowing_l3_l4_Moderate'
 'left_ne

In [4]:
import pydicom
import numpy as np
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import matplotlib.pyplot as plt

# Step 2: Define paths to the DICOM files and CSV file
root = '/kaggle/input/rsna-2024-lumbar-spine-degenerative-classification'
train_csv = '/kaggle/input/rsna-2024-lumbar-spine-degenerative-classification/train.csv'
dicom_main_dir = '/kaggle/input/rsna-2024-lumbar-spine-degenerative-classification/train_images/'
train_series_desc_dir = root + '/train_series_descriptions.csv'
# Step 3: Read the labels from the CSV file and one-hot encode them
train_df = pd.read_csv(train_csv).fillna("Normal/Mild")
x = train_df.iloc[:, 1:]
train_series_desc = pd.read_csv(train_series_desc_dir)

label_enc = OneHotEncoder()
sparse_matrix = label_enc.fit_transform(x)
dense_matrix = sparse_matrix.toarray()  # Convert sparse matrix to dense numpy array
all_labels = torch.tensor(dense_matrix, dtype=torch.float32).view(-1, 25, 3)  # Convert to PyTorch tensor
# Step 4: Define a custom dataset class
class MRIDataset(Dataset):
    def __init__(self, df, df_desc, dicom_main_dir, labels, w=256,h=256,max_length = 195, transform=None):
        self.df = df
        self.df_desc = df_desc
        self.dicom_main_dir = dicom_main_dir
        self.labels = labels
        self.transform = transform
        self.max_length = max_length
        self.w=w
        self.h=h
    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        img_list = [[],[],[]]
        index = {'Axial T2': 0, 'Sagittal T2/STIR': 1, 'Sagittal T1': 2 }
        study_id = self.df.iloc[idx]["study_id"]
        for _, series_dirs, _ in os.walk(os.path.join(self.dicom_main_dir, str(study_id))):
            for series in series_dirs:
                observations = self.df_desc[(self.df_desc["study_id"] == study_id) & (self.df_desc["series_id"] == np.int64(series))]
                if observations.empty:
                    continue
                observations = observations.iloc[0]
                description = observations["series_description"]
                path = os.listdir(os.path.join(self.dicom_main_dir, str(study_id), series))
                for file in path:
                    # All of the files in this loop should be in the same channel
                    dicom_data = pydicom.dcmread(os.path.join(self.dicom_main_dir, str(study_id), series, file))
                    image = dicom_data.pixel_array.astype(np.float32)
                    image = (image - np.min(image)) / (np.max(image) - np.min(image))  # Normalize to [0, 1]
                    if self.transform:
                        image = self.transform(image)
                        img_list[index[description]].append(image.squeeze())
            break
        padded_img_list, attention_masks = [], []
        for i in range(len(img_list)):
            padded_sequence, mask = self.add_padding_and_convert_to_torch(img_list[i], self.max_length, self.w, self.h)
            padded_img_list.append(padded_sequence)
            attention_masks.append(mask)
        label = self.labels[idx]
        padded_img_list = torch.stack(padded_img_list, dim=0)
        attention_masks = torch.stack(attention_masks, dim=0)

        return padded_img_list, attention_masks, label

    def add_padding_and_convert_to_torch(self, lst, max_length, w, h):
        if len(lst) < max_length:
            dim = (w,h)
            zero_matrix = torch.zeros(dim)
            mask = [1] * len(lst) + [0] * (max_length - len(lst))
            while len(lst) < max_length:
                lst.append(zero_matrix)
        else:
            lst = lst[:max_length]
            mask = [1] * max_length
        lst = torch.stack(lst, dim=0)
        mask = torch.tensor(mask, dtype=torch.long)
        return lst, mask
#         except Exception as e:
#             print("Error: ", e)
#             print("Study ID: ", study_id)
#             print("Index: ", i)
#             print("Image List:", lst)
#             return [], []
# Define transformations (if needed)
transform = transforms.Compose([
    transforms.ToTensor(),  # Convert to tensor
    transforms.Resize((256, 256), antialias=None),
    transforms.Normalize((0.5,), (0.5,))  # Normalize to [-1, 1]
])


def display_one_study(dataloader):
    for batch in dataloader:
        print(batch[0].size())
        print(batch[1].size())
        print(batch[2].size())
#         for idx, series_data in batch:
#             label = labels[idx]        
#             for series in series_data:
#                 images = series["images"]
#                 series_id = series["series_id"]
#                 description = series["description"]
#                 print(f"Series ID: {series_id}")
#                 print(f"Description: {description}")

#                 num_images = len(images)
#                 num_cols = 4  # Number of columns in the grid
#                 num_rows = (num_images + num_cols - 1) // num_cols  # Compute number of rows
#                 plt.figure(figsize=(num_cols * 4, num_rows * 4))
#                 for i, image in enumerate(images):
#                     image = image.squeeze()  # This will remove the extra dimensions
#                     plt.subplot(num_rows, num_cols, i + 1)
#                     plt.imshow(image, cmap='gray')
#                     plt.axis('off')
#     #             plt.suptitle(f'Label: {label.item()}')  # Display the label as the title
#                 plt.show()

            # break  # Only show one study for illustration purposes
          # Only iterate once for illustration purposes


In [5]:
train_series_desc = pd.read_csv(train_series_desc_dir)
print(train_series_desc['study_id'].nunique())
train = pd.read_csv(train_csv)
print(len(train))

1975
1975


In [ ]:
|

In [6]:
# Create the dataset and dataloader
train_dataset = MRIDataset(train_df, train_series_desc, dicom_main_dir, all_labels, transform=transform)
train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True)
# display_one_study(train_loader)


In [7]:
torch.cuda.is_available()
def get_default_device():
    """Pick GPU if available, else CPU"""
    if torch.cuda.is_available():
        print("GPU Training")
        return torch.device('cuda')
    else:
        return torch.device('cpu')
def to_device(data, device):
    """Move tensor(s) to chosen device"""
    if isinstance(data, (list,tuple)):
        return [to_device(x, device) for x in data]
    return data.to(device, non_blocking=True)
device = get_default_device()
torch.device('cuda')

GPU Training


device(type='cuda')

In [34]:
# Checking memory
import gc
# # Manually run garbage collector
gc.collect()

# # Empty CUDA cache
torch.cuda.empty_cache()
print(f"Allocated Memory before loading: {torch.cuda.memory_allocated('cuda') / 1024**2:.2f} MB")
print(f"Reserved Memory before loading: {torch.cuda.memory_reserved('cuda') / 1024**2:.2f} MB")
train_dataset = MRIDataset(train_df, train_series_desc, dicom_main_dir, all_labels, transform=transform)
train_loader = DataLoader(train_dataset, batch_size=1, shuffle=True)

for batch in train_loader:
    images = batch[0]
    labels = batch[2]
    to_device(images, device)
    to_device(labels, device)
    print(images.size())
    del images
    del labels
    break
# Print memory usage
print(f"Allocated Memory: {torch.cuda.memory_allocated('cuda') / 1024**2:.2f} MB")
print(f"Reserved Memory: {torch.cuda.memory_reserved('cuda') / 1024**2:.2f} MB")

# Delete tensors


# # Manually run garbage collector
gc.collect()

# # Empty CUDA cache
torch.cuda.empty_cache()

# Print memory usage after clearing
print(f"Allocated Memory after clearing: {torch.cuda.memory_allocated('cuda') / 1024**2:.2f} MB")
print(f"Reserved Memory after clearing: {torch.cuda.memory_reserved('cuda') / 1024**2:.2f} MB")

Allocated Memory before loading: 174.10 MB
Reserved Memory before loading: 2192.00 MB
torch.Size([1, 3, 195, 256, 256])
Allocated Memory: 174.10 MB
Reserved Memory: 2192.00 MB
Allocated Memory after clearing: 174.10 MB
Reserved Memory after clearing: 2192.00 MB


248

May need to add more processing of images, check distribution of data also
(Data Augmentation?)

In [8]:
import os
def get_shortest_video(directory):
    shortest = 1000000
    for study in os.listdir(directory):
        for series in os.listdir(os.path.join(directory, study)):
            length = len(os.listdir(os.path.join(directory, study, series)))
            if length < shortest:
                shortest = length
                print(os.path.join(directory, study), length)
    return shortest

# get_shortest_video("/kaggle/input/rsna-2024-lumbar-spine-degenerative-classification/test_images")

import os
def get_longest_video(directory):
    longest = 0
    for study in os.listdir(directory):
        for series in os.listdir(os.path.join(directory, study)):
            length = len(os.listdir(os.path.join(directory, study, series)))
            if length > longest:
                longest = length
                print(os.path.join(directory, study, series), length)
    return longest

# get_longest_video("/kaggle/input/rsna-2024-lumbar-spine-degenerative-classification/train_images") # Shortest: 2

In [9]:
def display_from_study(directory):
    images = []
    titles = []

    for series in os.listdir(directory):
        series_path = os.path.join(directory, series)
        if not os.path.isdir(series_path):
            continue  # Skip if it's not a directory
#         if not (series in ['300517765', '2097107888', '2679683906', '3114813181']):
#             continue
        for file in os.listdir(series_path):
            print(os.listdir(series_path))
            file_path = os.path.join(series_path, file)
            dicom_data = pydicom.dcmread(file_path)
            image = dicom_data.pixel_array.astype(np.float32)

            images.append(image)
            titles.append(f'Series: {series}, File: {file}')
        
    num_images = len(images)
    num_cols = 4  # Number of columns in the grid
    num_rows = (num_images + num_cols - 1) // num_cols  # Compute number of rows
    plt.figure(figsize=(num_cols * 4, num_rows * 4))

    for i, (image, title) in enumerate(zip(images, titles)):
        plt.subplot(num_rows, num_cols, i + 1)
        plt.imshow(image, cmap='gray')
        plt.axis('off')
    plt.show()


directory = '/kaggle/input/rsna-2024-lumbar-spine-degenerative-classification/train_images/4096820034'
# display_from_study(directory)

In [10]:
import torch
from torch import nn

from einops import rearrange, repeat, reduce
from einops.layers.torch import Rearrange

# helpers

def exists(val):
    return val is not None

def pair(t):
    return t if isinstance(t, tuple) else (t, t)

# classes

class FeedForward(nn.Module):
    def __init__(self, dim, hidden_dim, dropout = 0.):
        super().__init__()
        self.net = nn.Sequential(
            nn.LayerNorm(dim),
            nn.Linear(dim, hidden_dim),
            nn.GELU(),
            nn.Dropout(dropout),
            nn.Linear(hidden_dim, dim),
            nn.Dropout(dropout)
        )
    def forward(self, x):
        return self.net(x)

class Attention(nn.Module):
    def __init__(self, dim, heads = 8, dim_head = 64, dropout = 0.):
        super().__init__()
        inner_dim = dim_head *  heads
        project_out = not (heads == 1 and dim_head == dim)

        self.heads = heads
        self.scale = dim_head ** -0.5

        self.norm = nn.LayerNorm(dim)
        self.attend = nn.Softmax(dim = -1)
        self.dropout = nn.Dropout(dropout)

        self.to_qkv = nn.Linear(dim, inner_dim * 3, bias = False)
        
        self.to_out = nn.Sequential(
            nn.Linear(inner_dim, dim),
            nn.Dropout(dropout)
        ) if project_out else nn.Identity()

    def forward(self, x, mask=None):
        x = self.norm(x)
        qkv = self.to_qkv(x).chunk(3, dim = -1)
        q, k, v = map(lambda t: rearrange(t, 'b n (h d) -> b h n d', h = self.heads), qkv)

        dots = torch.matmul(q, k.transpose(-1, -2)) * self.scale
#         print(dots.size())
#         if exists(mask):
#             mask = rearrange(mask, 'b n -> b 1 1 n')
#             print(mask.size())
#             dots = dots.masked_fill(mask == 0, float('-inf'))
        attn = self.attend(dots)
        attn = self.dropout(attn)

        out = torch.matmul(attn, v)
        out = rearrange(out, 'b h n d -> b n (h d)')
        return self.to_out(out)

class Transformer(nn.Module):
    def __init__(self, dim, depth, heads, dim_head, mlp_dim, dropout = 0.):
        super().__init__()
        self.norm = nn.LayerNorm(dim)
        self.layers = nn.ModuleList([])
        
        for _ in range(depth):
            self.layers.append(nn.ModuleList([
                Attention(dim, heads = heads, dim_head = dim_head, dropout = dropout),
                FeedForward(dim, mlp_dim, dropout = dropout)
            ]))
    def forward(self, x, mask=None):
        for attn, ff in self.layers:
            x = attn(x, mask=mask) + x
            x = ff(x) + x
        return self.norm(x)

class ViT(nn.Module):
    def __init__(
        self,
        *,
        image_size,
        image_patch_size,
        frames,
        frame_patch_size,
        num_classes,
        dim,
        spatial_depth,
        temporal_depth,
        heads,
        mlp_dim,
        pool = 'cls',
        channels = 3,
        dim_head = 64,
        dropout = 0.,
        emb_dropout = 0.
    ):
        super().__init__()
        image_height, image_width = pair(image_size)
        patch_height, patch_width = pair(image_patch_size)
        self.num_classes = num_classes
        assert image_height % patch_height == 0 and image_width % patch_width == 0, 'Image dimensions must be divisible by the patch size.'
        assert frames % frame_patch_size == 0, 'Frames must be divisible by frame patch size'

        num_image_patches = (image_height // patch_height) * (image_width // patch_width)
        num_frame_patches = (frames // frame_patch_size)

        patch_dim = channels * patch_height * patch_width * frame_patch_size

        assert pool in {'cls', 'mean'}, 'pool type must be either cls (cls token) or mean (mean pooling)'

        self.global_average_pool = pool == 'mean'
        self.to_patch_embedding = nn.Sequential(
            Rearrange('b c (f pf) (h p1) (w p2) -> b f (h w) (p1 p2 pf c)', p1 = patch_height, p2 = patch_width, pf = frame_patch_size),
            nn.LayerNorm(patch_dim),
            nn.Linear(patch_dim, dim),
            nn.LayerNorm(dim)
        )

        self.pos_embedding = nn.Parameter(torch.randn(1, num_frame_patches, num_image_patches, dim))
        self.dropout = nn.Dropout(emb_dropout)

        self.spatial_cls_token = nn.Parameter(torch.randn(1, 1, dim)) if not self.global_average_pool else None
        self.temporal_cls_token = nn.Parameter(torch.randn(1, 1, dim)) if not self.global_average_pool else None

        self.spatial_transformer = Transformer(dim, spatial_depth, heads, dim_head, mlp_dim, dropout)
        self.temporal_transformer = Transformer(dim, temporal_depth, heads, dim_head, mlp_dim, dropout)

        self.pool = pool
        self.to_latent = nn.Identity()

        self.mlp_head = nn.Linear(dim, num_classes)

    def forward(self, video, mask=None):
        x = self.to_patch_embedding(video)
        b, f, n, _ = x.shape
        x = x + self.pos_embedding[:, :f, :n]
        if exists(self.spatial_cls_token):
            spatial_cls_tokens = repeat(self.spatial_cls_token, '1 1 d -> b f 1 d', b = b, f = f)
            x = torch.cat((spatial_cls_tokens, x), dim = 2)
        
        x = self.dropout(x)

        x = rearrange(x, 'b f n d -> (b f) n d')
        mask = rearrange(mask, 'b n f -> (b n) f')
        # attend across space
        x = self.spatial_transformer(x, mask=mask)
        x = rearrange(x, '(b f) n d -> b f n d', b = b)

        # excise out the spatial cls tokens or average pool for temporal attention

        x = x[:, :, 0] if not self.global_average_pool else reduce(x, 'b f n d -> b f d', 'mean')

        # append temporal CLS tokens

        if exists(self.temporal_cls_token):
            temporal_cls_tokens = repeat(self.temporal_cls_token, '1 1 d-> b 1 d', b = b)

            x = torch.cat((temporal_cls_tokens, x), dim = 1)
        
        # attend across time

        x = self.temporal_transformer(x)
        # excise out temporal cls token or average pool

        x = x[:, 0] if not self.global_average_pool else reduce(x, 'b f d -> b d', 'mean')

        x = self.to_latent(x)
        return self.mlp_head(x)

In [11]:
class VitLumbarSpine(nn.Module):
    def __init__(self, vit, num_diseases=5, num_areas=5, num_classes=3):
        super(VitLumbarSpine, self).__init__()
        self.vit = vit
        self.num_diseases = num_diseases
        self.num_areas = num_areas
        vit_output_dim = vit.num_classes

        # Create separate output heads for each disease in each area
        self.outputs = nn.ModuleList()
        for _ in range(num_diseases * num_areas): # should be 25
            self.outputs.append(nn.Linear(vit_output_dim, num_classes))
    
    def forward(self, x, mask=None):
        features = self.vit(x, mask)
        outputs = [output(features) for output in self.outputs]
        return outputs

In [18]:
from tqdm import tqdm
def train(model, train_loader, criterion, optimizer, epochs=5, save_path='./vivit_checkpoints', save_freq=1, device=get_default_device()):
    model.train()
#     print(f"Allocated Memory before loading: {torch.cuda.memory_allocated('cuda') / 1024**2:.2f} MB")
#     print(f"Reserved Memory before loading: {torch.cuda.memory_reserved('cuda') / 1024**2:.2f} MB")
    os.makedirs(save_path, exist_ok=True)
    for epoch in range(epochs):
        running_loss = 0.0
        # Initialize tqdm for progress bar
        pbar = tqdm(enumerate(train_loader), total=len(train_loader))
        for batch_idx, (images, masks, labels) in pbar:
            optimizer.zero_grad()
            images = to_device(images, device)
            labels = to_device(labels, device)
#             print(f"Allocated Memory after loading new batch: {torch.cuda.memory_allocated('cuda') / 1024**2:.2f} MB")
#             print(f"Reserved Memory after loading new batch: {torch.cuda.memory_reserved('cuda') / 1024**2:.2f} MB")
            # Forward pass
            outputs = model(images, masks)
#             print(f"Allocated Memory after having run batch: {torch.cuda.memory_allocated('cuda') / 1024**2:.2f} MB")
#             print(f"Reserved Memory after run batch: {torch.cuda.memory_reserved('cuda') / 1024**2:.2f} MB")
            # Compute the loss for each output
            loss = 0
            for i in range(len(outputs)):
                loss += criterion(outputs[i], labels[:, i])
            
            # Backward pass and optimization
            loss.backward()
            optimizer.step()
            
            running_loss += loss.item()
            
            # Update tqdm progress bar
            pbar.set_description(f"Epoch [{epoch+1}/{epochs}], Loss: {running_loss/(batch_idx+1):.4f}")
#             print(f"Allocated Memory after loss for batch: {torch.cuda.memory_allocated('cuda') / 1024**2:.2f} MB")
#             print(f"Reserved Memory after loss for batch: {torch.cuda.memory_reserved('cuda') / 1024**2:.2f} MB")
        # Print epoch summary
        print(f"Epoch [{epoch+1}/{epochs}], Loss: {running_loss/len(train_loader):.4f}")
        
        # Save model checkpoint
        if (epoch + 1) % save_freq == 0:
            checkpoint = {
                'epoch': epoch + 1,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'loss': running_loss / len(train_loader)
            }
            checkpoint_path = os.path.join(save_path, f'checkpoint2_epoch_{epoch + 1}.pt')
            torch.save(checkpoint, checkpoint_path)
            print(f"Checkpoint saved at {checkpoint_path}")

GPU Training


In [26]:
torch.cuda.empty_cache()

In [31]:
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "max_split_size_mb:128"

In [19]:
import os
print(os.getenv('PYTORCH_CUDA_ALLOC_CONF'))  # Should output: 'max_split_size_mb:128'

None


In [19]:
# # Train the model
vit_model = ViT(image_size=(256,256),
        image_patch_size=16,
        frames=195,
        frame_patch_size=5,
        num_classes=256,
        dim=512,
        spatial_depth=8,
        temporal_depth=8,
        heads=12,
        mlp_dim=512,)
initial_memory = torch.cuda.memory_allocated(device)
vit_model = to_device(vit_model, device)
model = VitLumbarSpine(vit_model)
model = to_device(model, device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

final_memory = torch.cuda.memory_allocated(device)
model_memory = final_memory - initial_memory
print(f"Memory allocated for the model: {model_memory / 1024**2:.2f} MB")
print(f"GPU used {torch.cuda.memory_allocated(device=get_default_device()) / 1024**2:.2f} MB of memory")

train(model, train_loader, criterion, optimizer, epochs=10)


Memory allocated for the model: 157.85 MB
GPU Training
GPU used 13316.17 MB of memory


  0%|          | 0/988 [00:03<?, ?it/s]


OutOfMemoryError: CUDA out of memory. Tried to allocate 236.00 MiB. GPU 0 has a total capacty of 15.89 GiB of which 175.12 MiB is free. Process 3105 has 15.71 GiB memory in use. Of the allocated memory 15.18 GiB is allocated by PyTorch, and 241.01 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [1]:
!ls

In [46]:
def test(model, test_loader):
    model.eval()
    correct = [0] * (num_diseases * num_areas)
    total = 0
    
    with torch.no_grad():
        for images, labels in test_loader:
            outputs = model(images)
            for i in range(len(outputs)):
                _, predicted = torch.max(outputs[i], 1)
                correct[i] += (predicted == labels[:, i]).sum().item()
            total += labels.size(0)
    
    for i in range(len(correct)):
        print(f'Accuracy of output {i} on the test images: {100 * correct[i] / total:.2f}%')
# test(model, test_loader)

/kaggle/working


In [92]:
!cd vivit_checkpoints/; pwd

/kaggle/working/vivit_checkpoints
